In [ ]:
# NOTE line_profiler
#   - This __builtins__.profile makes `profile` available everywhere, so modules can e.g. `@profile` their functions
#       - You can also profile.add_function(f) here instead of having to @profile deep in (someone else's) code
#   - Very tricky!
# import line_profiler
# __builtins__.profile = line_profiler.LineProfiler()

In [ ]:
from notebooks import *

In [ ]:
load = Load()
# with cache_control():  # FIXME Hangs without refresh=True [why?] -- which fortunately doesn't replay the slow .mp3->.wav step
# with cache_control(refresh=True):
with cache_control(refresh=False):
    recs_all = load.recs(
        datasets=['xc'],
    )
display(
    df_summary(recs_all).T,
    # df_summary(recs_all),
    recs_all[:5],
)

•

,,,,,,,,min,25%,50%,75%,max
,dtype,sizeof,len,count,nunique,mean,std,,,,,
dataset,object,701760,13760,13760,1,NaN,NaN,xc,xc,xc,xc,xc
species,category,729280,13760,13760,168,NaN,NaN,MOUQ,CORA,RCKI,WCSP,HOSP
species_com_name,category,886706,13760,13760,168,NaN,NaN,Mountain Quail,Common Raven,Ruby-crowned Kinglet,White-crowned Sparrow,House Sparrow
species_query,object,729280,13760,13760,168,NaN,NaN,ACWO,CATH,HOSP,RUHU,YTWA
duration_s,float64,330240,13760,13760,10242,6.323e+01,8.120e+01,0.216,20.1,39.9,75.8,2.35e+03
samples_mb,float64,330240,13760,13760,10242,2.659e+00,3.415e+00,0.00908,0.844,1.68,3.19,99
samples_n,int64,385280,13760,13760,10242,1.394e+06,1.790e+06,4763,442534,880704,1670976,51919071
basename,object,743040,13760,13760,1,NaN,NaN,audio,audio,audio,audio,audio
species_longhand,category,983026,13760,13760,168,NaN,NaN,Mountain Quail - MOUQ,Common Raven - CORA,Ruby-crowned Kinglet - RCKI,White-crowned Sparrow - WCSP,House Sparrow - HOSP


,dataset,species,species_com_name,species_query,duration_s,samples_mb,samples_n,basename,species_longhand,id,path,filesize_b
0,xc,MOUQ,Mountain Quail,MOUQ,147.0,6.180,3241728,audio,Mountain Quail - MOUQ,xc/data/MOUQ/72099/audio,xc/data/MOUQ/72099/audio.mp3,2357229
1,xc,MOUQ,Mountain Quail,MOUQ,70.9,2.980,1562669,audio,Mountain Quail - MOUQ,xc/data/MOUQ/239499/audio,xc/data/MOUQ/239499/audio.mp3,1140133
2,xc,MOUQ,Mountain Quail,MOUQ,15.6,0.656,343896,audio,Mountain Quail - MOUQ,xc/data/MOUQ/313537/audio,xc/data/MOUQ/313537/audio.mp3,299858
3,xc,MOUQ,Mountain Quail,MOUQ,48.2,2.030,1062163,audio,Mountain Quail - MOUQ,xc/data/MOUQ/329439/audio,xc/data/MOUQ/329439/audio.mp3,776854
4,xc,MOUQ,Mountain Quail,MOUQ,41.4,1.740,912867,audio,Mountain Quail - MOUQ,xc/data/MOUQ/329454/audio,xc/data/MOUQ/329454/audio.mp3,668764


In [ ]:
# Subset the data
#   - Runtime: reduce data volume
#   - Class imbalance: clip all classes at the same N (or smaller)
# class_size = np.inf  # recs[14k]  @ species[168]
# class_size = 200     # recs[12k]  @ species[168]
# class_size = 100     # recs[10k]  @ species[168]
# class_size = 50      # recs[7.0k] @ species[168]
class_size = 20      # recs[3.2k] @ species[168]
# class_size = 10      # recs[1.7k] @ species[168]
# class_size = 5       # recs[.84k] @ species[168]
# class_size = 2       # recs[.34k] @ species[168]
recs = (recs_all
    .groupby('species')
    .apply(lambda g: g.sample(n=min(len(g), class_size), random_state=0))
    # Drop classes with <2 instances, else StratifiedShuffleSplit complains (e.g. 'TUVU')
    [lambda df: df.species.isin(df.species.value_counts()[lambda s: s >= 2].index)]
)
display(
    df_summary(recs_all).T,
    df_summary(recs).T,
)

,,,,,,,,min,25%,50%,75%,max
,dtype,sizeof,len,count,nunique,mean,std,,,,,
dataset,object,866880,13760,13760,1,NaN,NaN,xc,xc,xc,xc,xc
species,category,729280,13760,13760,168,NaN,NaN,MOUQ,CORA,RCKI,WCSP,HOSP
species_com_name,category,886706,13760,13760,168,NaN,NaN,Mountain Quail,Common Raven,Ruby-crowned Kinglet,White-crowned Sparrow,House Sparrow
species_query,object,729380,13760,13760,168,NaN,NaN,ACWO,CATH,HOSP,RUHU,YTWA
duration_s,float64,330240,13760,13760,10242,6.323e+01,8.120e+01,0.216,20.1,39.9,75.8,2.35e+03
samples_mb,float64,330240,13760,13760,10242,2.659e+00,3.415e+00,0.00908,0.844,1.68,3.19,99
samples_n,int64,385280,13760,13760,10242,1.394e+06,1.790e+06,4763,442534,880704,1670976,51919071
basename,object,743160,13760,13760,1,NaN,NaN,audio,audio,audio,audio,audio
species_longhand,category,983026,13760,13760,168,NaN,NaN,Mountain Quail - MOUQ,Common Raven - CORA,Ruby-crowned Kinglet - RCKI,White-crowned Sparrow - WCSP,House Sparrow - HOSP


,,,,,,,,min,25%,50%,75%,max
,dtype,sizeof,len,count,nunique,mean,std,,,,,
dataset,object,204309,3243,3243,1,NaN,NaN,xc,xc,xc,xc,xc
species,category,171879,3243,3243,167,NaN,NaN,MOUQ,AMKE,HOWR,GCWA,HOSP
species_com_name,category,210978,3243,3243,167,NaN,NaN,Mountain Quail,American Kestrel,House Wren,Golden-cheeked Warbler,House Sparrow
species_query,object,171919,3243,3243,167,NaN,NaN,ACWO,CAQU,LASP,RTHA,YTWA
duration_s,float64,77832,3243,3243,2935,6.328e+01,8.494e+01,0.744,18.9,39.1,76.6,2.35e+03
samples_mb,float64,77832,3243,3243,2935,2.661e+00,3.572e+00,0.0313,0.797,1.64,3.22,99
samples_n,int64,90804,3243,3243,2935,1.395e+06,1.873e+06,16405,417600,861120,1688677,51919071
basename,object,175122,3243,3243,1,NaN,NaN,audio,audio,audio,audio,audio
species_longhand,category,233679,3243,3243,167,NaN,NaN,Mountain Quail - MOUQ,American Kestrel - AMKE,House Wren - HOWR,Golden-cheeked Warbler - GCWA,House Sparrow - HOSP


In [ ]:
# Load a saved projection model
projection = Projection.load('peterson-v0-26bae1c', features=Features(load=load))


[20:18:00.496] DEBUG  Projection.load
  path: /Users/danb/hack/bubo/data/models/projection/peterson-v0-26bae1c.pkl


In [ ]:
# Add .feat
recs = projection.transform(recs)

•

In [ ]:
# GridSearchCV many models / model params
param_grid = dict(
    classifier=[
        'cls: rf, random_state: 0, criterion: gini, n_estimators: 1', # 2.3s fit -> 3.7s cell
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 10',  # 6.8s fit -> 14s cell
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 100',  # 51s fit -> 112s cell
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 1000', # ?s fit -> ?s cell
    ]
)

cv = GridSearchCVCached(
    estimator=Search(projection=projection, classifier='DUMMY_CLASSIFIER'),
    param_grid=param_grid,
    # NOTE line_profiler
    #   - The outcome of doing the line profiling was discovering that refit=True was on by default :/
    # refit=False,  # Don't spend time fitting cv.best_estimator_ at the end (default: True)
    # cv=3,  # [SP14] uses two-/three-fold CV [why?]
    # Stratified ensures that all classes have >0 instances in each split, which is statistically maybe fishy but avoids
    # breaking various code that merges the split results back together and assumes all splits' classes are the same
    cv=sk.model_selection.StratifiedShuffleSplit(
        n_splits=1,  # [for ~1/2 hit/miss: ~9m, ~10g disk cache]
        # n_splits=2,
        # n_splits=5,
        # n_splits=10,
        # n_splits=20,  # Known good [>51m uncached, >25g disk cache]
        # n_splits=100,  # [?m runtime, ?g disk cache]
        test_size=.2,
        random_state=0,
    ),
    return_train_score=True,
    extra_metrics=dict(
        # [How to specify SearchEvals here without creating caching headaches?]
        #   - e.g. defs don't bust cache on code edit
        #   - And avoid thrashing cache every time we refactor SearchEvals
        test_i='i_test',
        test_y='y_test',
        test_classes='estimator.classes_',
        test_predict_proba='estimator.classifier_.predict_proba(X_test)',
    ),
    # return_estimator=True,  # Verrrry heavy, use extra_metrics instead
    # verbose=100,
    # verbose=10,  # O(models * n_splits) lines of outputs
    verbose=1,  # O(1) lines of outputs
    # n_jobs=1,  # For %prun
    # n_jobs=8,
    n_jobs=16,
)
with contextlib.ExitStack() as stack:
    # stack.enter_context(cache_control(refresh=True))
    stack.enter_context(cache_control(enabled=False))
    # stack.enter_context(joblib.parallel_backend('threading'))  # Default: 'multiprocessing'
    # stack.enter_context(joblib.parallel_backend('sequential'))  # For %prun [FIXME Has no effect; why?]
    stack.enter_context(log.context(level='info'))
    X, y = Search.Xy(recs)

    # cv.fit(X, y)

    # NOTE line_profiler
    #   - `%lprun` creates its own LineProfiler(), which we don't want (we already created one above), so use vanilla
    #     profile.runcall/print_stats instead
    #   - This requires reading the code to map the %lprun args (e.g. -u -> output_unit)
    #       - https://github.com/rkern/line_profiler/blob/master/line_profiler.py#L264-L394
    #   - WARNING Reusing the same LineProfiler instance causes its stats to accumulate
    profile = line_profiler.LineProfiler()
    profile.add_function(GridSearchCVCached.fit)
    profile.runcall(lambda: cv.fit(X, y))
    profile.print_stats(output_unit=1)

Fitting 1 folds for each of 1 candidates, totalling 1 fits


[20:18:01.322] INFO   _fit_and_score... split_i[0], estimator[Search(classifier='DUMMY_CLASSIFIER')]


[20:18:04.145] INFO   _fit_and_score. split_i[0], estimator[Search(classifier='cls: rf, random_state: 0, criterion: gini, n_estimators: 1')]


[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    3.1s finished


Timer unit: 1 s

Total time: 4.28646 s
File: /Users/danb/hack/bubo/features/sk_util.py
Function: fit at line 145

Line #      Hits         Time  Per Hit   % Time  Line Contents
   145                                               def fit(self, X, y=None, groups=None, **fit_params):
   146                                                   """Run fit with all sets of parameters.
   147                                           
   148                                                   Parameters
   149                                                   ----------
   150                                           
   151                                                   X : array-like, shape = [n_samples, n_features]
   152                                                       Training vector, where n_samples is the number of samples and
   153                                                       n_features is the number of features.
   154                                           
   155 